In [1]:
!pip install "polars[all]"

   ---------------------------------------- 0.0/731.2 kB ? eta -:--:--
   ---------------------------------------- 731.2/731.2 kB 10.0 MB/s  0:00:00
   ---------------------------------------- 0.0/43.9 MB ? eta -:--:--
   -- ------------------------------------- 3.1/43.9 MB 15.4 MB/s eta 0:00:03
   ------ --------------------------------- 7.6/43.9 MB 20.4 MB/s eta 0:00:02
   --------- ------------------------------ 10.0/43.9 MB 16.3 MB/s eta 0:00:03
   ------------ --------------------------- 14.2/43.9 MB 16.8 MB/s eta 0:00:02
   ---------------- ----------------------- 18.1/43.9 MB 17.5 MB/s eta 0:00:02
   --------------------- ------------------ 23.1/43.9 MB 18.5 MB/s eta 0:00:02
   ------------------------- -------------- 27.8/43.9 MB 19.4 MB/s eta 0:00:01
   ----------------------------- ---------- 32.8/43.9 MB 19.8 MB/s eta 0:00:01
   ---------------------------------- ----- 37.7/43.9 MB 20.5 MB/s eta 0:00:01
   -------------------------------------- - 42.7/43.9 MB 20.9 MB/s eta 0

In [2]:
!wget https://github.com/akmtamimahmed/Lab-1.2/raw/refs/heads/main/data/lahman_1871-2024_csv/lahman_1871-2024_csv/Pitching.csv

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
!pip install more_polars


   ------------- -------------------------- 1/3 [composable]
   ---------------------------------------- 3/3 [more_polars]



In [31]:
import polars as pl
from pathlib import Path

In [ ]:
def dh_state_expr():
    
    return (
        pl.when(pl.col("yearID") < 1973)
        .then(pl.lit("No DH (both)"))
        .when((pl.col("yearID").is_between(1973, 2019)) & (pl.col("lgID") == "AL"))
        .then(pl.lit("DH in AL; NL no DH"))
        .when((pl.col("yearID").is_between(1973, 2019)) & (pl.col("lgID") == "NL"))
        .then(pl.lit("NL no DH"))
        .when(pl.col("yearID") == 2020)
        .then(pl.lit("Universal DH (2020)"))
        .when((pl.col("yearID") == 2021) & (pl.col("lgID") == "AL"))
        .then(pl.lit("DH in AL; NL no DH"))
        .when((pl.col("yearID") == 2021) & (pl.col("lgID") == "NL"))
        .then(pl.lit("NL no DH"))
        .when(pl.col("yearID") >= 2022)
        .then(pl.lit("Universal DH (2022+)"))
        .otherwise(pl.lit("Unknown"))
        .alias("DH_state")
    )

In [ ]:
# --- Task 1: ERA Analysis ---
pitching_url = 'https://github.com/akmtamimahmed/Lab-1.2/raw/main/data/lahman_1871-2024_csv/lahman_1871-2024_csv/Pitching.csv'
pitch_df = pl.read_csv(pitching_url)
pitch_modern = pitch_df.filter(pl.col("yearID") >= 1946)

era_group = (
    pitch_modern
    .group_by(["yearID", "lgID"])
    .agg([
        pl.col("ER").sum().alias("total_ER"),
        pl.col("IPouts").sum().alias("total_IPouts"),
    ])
    .with_columns(
        (pl.col("total_ER") / pl.col("total_IPouts") * 27).alias("ERA")
    )
)
era_group.head()

yearID,lgID,total_ER,total_IPouts,ERA
i64,str,i64,i64,f64
1965,"""AL""",5587,43618,3.458412
2005,"""NL""",10833,69157,4.229377
2014,"""NL""",8878,65445,3.662709
1956,"""AL""",5098,33095,4.159118
1996,"""AL""",11260,60815,4.999096


In [ ]:
# --- Task 2: Slugging Percentage Analysis ---
batting_url = 'https://github.com/akmtamimahmed/Lab-1.2/raw/main/data/lahman_1871-2024_csv/lahman_1871-2024_csv/Batting.csv'
bat_df = pl.read_csv(batting_url)
bat_modern = bat_df.filter(pl.col("yearID") >= 1946)

bat_group = (
    bat_modern
    .group_by(["yearID", "lgID"])
    .agg([
        pl.col("AB").sum().alias("total_AB"),
        pl.col("H").sum().alias("total_H"),
        pl.col("2B").sum().alias("total_2B"),
        pl.col("3B").sum().alias("total_3B"),
        pl.col("HR").sum().alias("total_HR"),
    ])
    .with_columns(
        (pl.col("total_H") + pl.col("total_2B") + 2 * pl.col("total_3B") + 3 * pl.col("total_HR")).alias("total_TB")
    )
    .with_columns(
        (pl.col("total_TB") / pl.col("total_AB")).alias("SLG")
    )
)
bat_group.head()

yearID,lgID,total_AB,total_H,total_2B,total_3B,total_HR,total_TB,SLG
i64,str,i64,i64,i64,i64,i64,i64,f64
2020,"""NL""",29399,7245,1402,126,1161,12382,0.421171
1997,"""NL""",77203,20300,3907,485,2163,31666,0.410165
2011,"""NL""",88119,22263,4394,487,2281,34474,0.391221
1964,"""NL""",55284,14032,2161,427,1211,20680,0.374068
1965,"""NL""",55377,13794,2122,422,1318,20714,0.374054


In [36]:
# --- Add DH State and Write to CSV ---
era_group = era_group.with_columns(dh_state_expr())
bat_group = bat_group.with_columns(dh_state_expr())

era_group.write_csv("league_era_by_year.csv")
bat_group.write_csv("league_slugging_by_year.csv")

print("\nSuccessfully created 'league_era_by_year.csv' and 'league_slugging_by_year.csv'")


Successfully created 'league_era_by_year.csv' and 'league_slugging_by_year.csv'


In [37]:
pl.read_csv("league_era_by_year.csv").head()

yearID,lgID,total_ER,total_IPouts,ERA,DH_state
i64,str,i64,i64,f64,str
1965,"""AL""",5587,43618,3.458412,"""No DH (both)"""
2005,"""NL""",10833,69157,4.229377,"""NL no DH"""
2014,"""NL""",8878,65445,3.662709,"""NL no DH"""
1956,"""AL""",5098,33095,4.159118,"""No DH (both)"""
1996,"""AL""",11260,60815,4.999096,"""DH in AL; NL no DH"""


In [ ]:
pl.read_csv("league_slugging_by_year.csv").head() 

yearID,lgID,total_AB,total_H,total_2B,total_3B,total_HR,total_TB,SLG,DH_state
i64,str,i64,i64,i64,i64,i64,i64,f64,str
2020,"""NL""",29399,7245,1402,126,1161,12382,0.421171,"""Universal DH (2020)"""
1997,"""NL""",77203,20300,3907,485,2163,31666,0.410165,"""NL no DH"""
2011,"""NL""",88119,22263,4394,487,2281,34474,0.391221,"""NL no DH"""
1964,"""NL""",55284,14032,2161,427,1211,20680,0.374068,"""No DH (both)"""
1965,"""NL""",55377,13794,2122,422,1318,20714,0.374054,"""No DH (both)"""
